In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [4]:
# create arangoDB database, or open it if it already exists
try:
    corp_db = conn.createDatabase(name="corp_data")
    # from command line:
    # arangoimport --file '''path to CorpData.csv''' --collection corp_data --create-collection true --type csv --server.database corp_data
    # arangoimport --file '''path to CorpIndividualExport.csv''' --collection corp_individual --create-collection true --type csv --server.database corp_data

except:
    corp_db = conn["corp_data"]

In [5]:
# extracting the values of the l3_assess collection
val_aql = "FOR x IN l3_assess RETURN x"
value_query_result = corp_db.AQLQuery(val_aql,rawResults=True, batchSize = 1000)
col_value = {}
ind_val = 0

for value in value_query_result:
    col_value[ind_val] = value
    ind_val += 1
    
# create dataframe from dictionary of dictionaries
l3 = pd.DataFrame.from_dict(data = col_value, orient = 'index')
l3_arr = l3.to_numpy()

_key                                        28345021
_id                               l3_assess/28345021
_rev                                     _eav8fHe---
PROP_ID                                    287-17-25
LOC_ID                              F_760330_2945596
PropAddr      99 POND AVE 606 P584B, BROOKLINE 02445
UNITS                                            1.0
OWNER1                             RIVENSON HOWARD L
OwnAddr      99 POND AVE, # 606, BROOKLINE, MA 02445
TOTAL_VAL                                     636700
FY                                              2021
LS_DATE                               840412800000.0
LS_PRICE                                    180000.0

7: OWNER1
8: OwnAddr

In [6]:
# extracting the values of the corp_data_processed collection
val_aql = "FOR x IN corp_data_processed RETURN x"
value_query_result = corp_db.AQLQuery(val_aql,rawResults=True, batchSize = 1000)
col_value = {}
ind_val = 0

for value in value_query_result:
    col_value[ind_val] = value
    ind_val += 1
    
# create dataframe from dictionary of dictionaries
corp = pd.DataFrame.from_dict(data = col_value, orient = 'index')
corp_arr = corp.to_numpy()

['17777999' 'corp_data_processed/17777999' '_eZCy3XK---' '00wepr' 'Domestic Limited Liability Company (LLC)' 'MILLER PLACE LLC' '189 WELLS AVE, STE 302, NEWTON CENTRE, MA 02459' 'BARRY S FOX' '189 WELLS AVE, STE 302, NEWTON CENTRE, MA 02459']

0: _key 1: _id 2: _rev 3: DataID 4: EntityType 5: EntityName 6: EntityAddress 7: AgentName 8: AgentAddress

In [39]:
def geo2corp_add(l3_arr, corp_arr, batch_size):
    
    num_batch = int(corp_df.shape[0] / batch_size)
    
    for i in range(0,num_batch+1):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_arr = l3_arr[i*batch_size:(i+1)*batch_size][:]
            
        for j in range(0,batch_arr.shape[0]):
            own_add = batch_arr[j][8] 
            l3_id = batch_arr[j][1]
            corp_add = np.where(corp_arr[:,6] == own_add)[0]
            corp_id = corp_arr[corp_add][:,1]
            for i_id in corp_id:
                edge_attributes = {"_from": l3_id, "_to": i_id, "type": 'geo2corp/address'}
                edge = corp_db["geo2corp"].createDocument(edge_attributes)
                edge.save()

geo2corp_add(l3_arr, corp_arr,1000)

1000
1000000 1001000
1001
1001000 1002000
1002
1002000 1003000
1003
1003000 1004000
1004
1004000 1005000
1005
1005000 1006000
1006
1006000 1007000
1007
1007000 1008000
1008
1008000 1009000
1009
1009000 1010000
1010
1010000 1011000
1011
1011000 1012000
1012
1012000 1013000
1013
1013000 1014000
1014
1014000 1015000
1015
1015000 1016000
1016
1016000 1017000
1017
1017000 1018000
1018
1018000 1019000
1019
1019000 1020000
1020
1020000 1021000
1021
1021000 1022000
1022
1022000 1023000
1023
1023000 1024000
1024
1024000 1025000
1025
1025000 1026000
1026
1026000 1027000
1027
1027000 1028000
1028
1028000 1029000
1029
1029000 1030000
1030
1030000 1031000
1031
1031000 1032000
1032
1032000 1033000
1033
1033000 1034000
1034
1034000 1035000
1035
1035000 1036000
1036
1036000 1037000
1037
1037000 1038000
1038
1038000 1039000
1039
1039000 1040000
1040
1040000 1041000
1041
1041000 1042000
1042
1042000 1043000
1043
1043000 1044000
1044
1044000 1045000
1045
1045000 1046000
1046
1046000 1047000
1047
1047000 

1391
1391000 1392000
1392
1392000 1393000
1393
1393000 1394000
1394
1394000 1395000
1395
1395000 1396000
1396
1396000 1397000
1397
1397000 1398000
1398
1398000 1399000
1399
1399000 1400000
1400
1400000 1401000
1401
1401000 1402000
1402
1402000 1403000
1403
1403000 1404000
1404
1404000 1405000
1405
1405000 1406000
1406
1406000 1407000
1407
1407000 1408000
1408
1408000 1409000
1409
1409000 1410000
1410
1410000 1411000
1411
1411000 1412000
1412
1412000 1413000
1413
1413000 1414000
1414
1414000 1415000
1415
1415000 1416000
1416
1416000 1417000
1417
1417000 1418000
1418
1418000 1419000
1419
1419000 1420000
1420
1420000 1421000
1421
1421000 1422000
1422
1422000 1423000
1423
1423000 1424000
1424
1424000 1425000
1425
1425000 1426000
1426
1426000 1427000
1427
1427000 1428000
1428
1428000 1429000
1429
1429000 1430000
1430
1430000 1431000
1431
1431000 1432000
1432
1432000 1433000
1433
1433000 1434000
1434
1434000 1435000
1435
1435000 1436000
1436
1436000 1437000
1437
1437000 1438000
1438
1438000 

1782
1782000 1783000
1783
1783000 1784000
1784
1784000 1785000
1785
1785000 1786000
1786
1786000 1787000
1787
1787000 1788000
1788
1788000 1789000
1789
1789000 1790000
1790
1790000 1791000
1791
1791000 1792000
1792
1792000 1793000
1793
1793000 1794000
1794
1794000 1795000
1795
1795000 1796000
1796
1796000 1797000
1797
1797000 1798000
1798
1798000 1799000
1799
1799000 1800000
1800
1800000 1801000
1801
1801000 1802000
1802
1802000 1803000
1803
1803000 1804000
1804
1804000 1805000
1805
1805000 1806000
1806
1806000 1807000
1807
1807000 1808000
1808
1808000 1809000
1809
1809000 1810000
1810
1810000 1811000
1811
1811000 1812000
1812
1812000 1813000
1813
1813000 1814000
1814
1814000 1815000
1815
1815000 1816000
1816
1816000 1817000
1817
1817000 1818000
1818
1818000 1819000
1819
1819000 1820000
1820
1820000 1821000
1821
1821000 1822000
1822
1822000 1823000
1823
1823000 1824000
1824
1824000 1825000
1825
1825000 1826000
1826
1826000 1827000
1827
1827000 1828000
1828
1828000 1829000
1829
1829000 

In [7]:
def geo2corp_own(l3_arr, corp_arr, batch_size):
    
    #num_batch = int(corp_df.shape[0] / batch_size)
    
    #for i in range(0,num_batch+1):
    for i in range(1366,2081):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_arr = l3_arr[i*batch_size:(i+1)*batch_size][:]
            
        for j in range(0,batch_arr.shape[0]):
            owner = batch_arr[j][7] 
            l3_id = batch_arr[j][1]
            corp_name = np.where(corp_arr[:,5] == owner)[0]
            corp_id = corp_arr[corp_name][:,1]
            for i_id in corp_id:
                edge_attributes = {"_from": l3_id, "_to": i_id, "type": 'geo2corp/entity_name'}
                edge = corp_db["geo2corp"].createDocument(edge_attributes)
                edge.save()

geo2corp_own(l3_arr, corp_arr,1000)

1366
1366000 1367000
1367
1367000 1368000
1368
1368000 1369000
1369
1369000 1370000
1370
1370000 1371000
1371
1371000 1372000
1372
1372000 1373000
1373
1373000 1374000
1374
1374000 1375000
1375
1375000 1376000
1376
1376000 1377000
1377
1377000 1378000
1378
1378000 1379000
1379
1379000 1380000
1380
1380000 1381000
1381
1381000 1382000
1382
1382000 1383000
1383
1383000 1384000
1384
1384000 1385000
1385
1385000 1386000
1386
1386000 1387000
1387
1387000 1388000
1388
1388000 1389000
1389
1389000 1390000
1390
1390000 1391000
1391
1391000 1392000
1392
1392000 1393000
1393
1393000 1394000
1394
1394000 1395000
1395
1395000 1396000
1396
1396000 1397000
1397
1397000 1398000
1398
1398000 1399000
1399
1399000 1400000
1400
1400000 1401000
1401
1401000 1402000
1402
1402000 1403000
1403
1403000 1404000
1404
1404000 1405000
1405
1405000 1406000
1406
1406000 1407000
1407
1407000 1408000
1408
1408000 1409000
1409
1409000 1410000
1410
1410000 1411000
1411
1411000 1412000
1412
1412000 1413000
1413
1413000 

1757
1757000 1758000
1758
1758000 1759000
1759
1759000 1760000
1760
1760000 1761000
1761
1761000 1762000
1762
1762000 1763000
1763
1763000 1764000
1764
1764000 1765000
1765
1765000 1766000
1766
1766000 1767000
1767
1767000 1768000
1768
1768000 1769000
1769
1769000 1770000
1770
1770000 1771000
1771
1771000 1772000
1772
1772000 1773000
1773
1773000 1774000
1774
1774000 1775000
1775
1775000 1776000
1776
1776000 1777000
1777
1777000 1778000
1778
1778000 1779000
1779
1779000 1780000
1780
1780000 1781000
1781
1781000 1782000
1782
1782000 1783000
1783
1783000 1784000
1784
1784000 1785000
1785
1785000 1786000
1786
1786000 1787000
1787
1787000 1788000
1788
1788000 1789000
1789
1789000 1790000
1790
1790000 1791000
1791
1791000 1792000
1792
1792000 1793000
1793
1793000 1794000
1794
1794000 1795000
1795
1795000 1796000
1796
1796000 1797000
1797
1797000 1798000
1798
1798000 1799000
1799
1799000 1800000
1800
1800000 1801000
1801
1801000 1802000
1802
1802000 1803000
1803
1803000 1804000
1804
1804000 

In [32]:
l3_arr.shape[0]

2079737

In [23]:
#for j in range(0,corp_arr.shape[0]):
for j in range(0,25):
    own = l3_arr[j][7] 
    #corp_id = batch_df[j][1]
    corp = np.where(corp_arr[:,5] == own)[0]
    #indiv_id = indiv_df[c_indiv][:,1]
    print(corp)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[919410]


In [25]:
own_arr = np.where(l3_arr[:,7] == corp_arr[919410][5])[0]
print(own_add)

[]


In [28]:
print(l3_arr[24,7])

WANNIAN INVESTMENT INC


In [29]:
print(corp_arr[919410][5])

WANNIAN INVESTMENT INC
